In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

/Users/JohnSchueler/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/JohnSchueler/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:366: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Users/JohnSchueler/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:366: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Users/JohnSchueler/opt/anaconda3/lib/python3.8/site-packages/dask/dataframe/util

In [2]:
#import data
train1 = pd.read_csv('train.csv', low_memory=False)

In [3]:
#drop columns
train0 = train1.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageCond', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'MasVnrType', 'MSSubClass', 'OverallCond', 'YearBuilt', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtHalfBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'GarageArea', 'Street', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'BsmtQual', 'YearRemodAdd', 'TotRmsAbvGrd', 'MiscVal', 'MoSold', 'YrSold', 'RoofStyle', 'ExterCond', 'PavedDrive', 'HeatingQC', 'Functional'])

In [4]:
train0.head(5)

,Id,MSZoning,LotFrontage,LotArea,LotShape,Neighborhood,HouseStyle,OverallQual,RoofMatl,Exterior1st,...,GarageQual,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,SaleType,SaleCondition,SalePrice
0,1,RL,65.0,8450,Reg,CollgCr,2Story,7,CompShg,VinylSd,...,TA,0,61,0,0,0,0,WD,Normal,208500
1,2,RL,80.0,9600,Reg,Veenker,1Story,6,CompShg,MetalSd,...,TA,298,0,0,0,0,0,WD,Normal,181500
2,3,RL,68.0,11250,IR1,CollgCr,2Story,7,CompShg,VinylSd,...,TA,0,42,0,0,0,0,WD,Normal,223500
3,4,RL,60.0,9550,IR1,Crawfor,2Story,7,CompShg,Wd Sdng,...,TA,0,35,272,0,0,0,WD,Abnorml,140000
4,5,RL,84.0,14260,IR1,NoRidge,2Story,8,CompShg,VinylSd,...,TA,192,84,0,0,0,0,WD,Normal,250000


In [5]:
train0.dtypes

Id                 int64
MSZoning          object
LotFrontage      float64
LotArea            int64
LotShape          object
Neighborhood      object
HouseStyle        object
OverallQual        int64
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrArea       float64
ExterQual         object
Foundation        object
TotalBsmtSF        int64
Heating           object
CentralAir        object
Electrical        object
GrLivArea          int64
BsmtFullBath       int64
FullBath           int64
KitchenQual       object
Fireplaces         int64
GarageCars         int64
GarageQual        object
WoodDeckSF         int64
OpenPorchSF        int64
EnclosedPorch      int64
3SsnPorch          int64
ScreenPorch        int64
PoolArea           int64
SaleType          object
SaleCondition     object
SalePrice          int64
dtype: object

In [6]:
#get dummies
train = pd.get_dummies(train0, columns=["MSZoning", "LotShape", "Neighborhood", "HouseStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "ExterQual", "Foundation", "Heating", "CentralAir", "Electrical", "KitchenQual", "GarageQual", "SaleType", "SaleCondition"])

In [7]:
train.dtypes

Id                         int64
LotFrontage              float64
LotArea                    int64
OverallQual                int64
MasVnrArea               float64
                          ...   
SaleCondition_AdjLand      uint8
SaleCondition_Alloca       uint8
SaleCondition_Family       uint8
SaleCondition_Normal       uint8
SaleCondition_Partial      uint8
Length: 146, dtype: object

In [8]:
# set to column "salesprice"
X, y = train.iloc[:,:-1], train.iloc[:,-1]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [10]:
DM_train = xgb.DMatrix(data=X_train, label=y_train)

/Users/JohnSchueler/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:173: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [11]:
DM_test = xgb.DMatrix(data=X_test, label=y_test)

In [12]:
params = {"booster":"gblinear", "objective":"reg:linear"}

In [13]:
xg_reg = xgb.train(params = params, dtrain=DM_train, num_boost_round=5)

[20:21:05] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[20:21:05] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


In [14]:
preds = xg_reg.predict(DM_test)

In [15]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.056839


In [16]:
mae = mean_absolute_error(y_test, preds)
print("MAE: %f" % (mae))

MAE: 0.044707


In [17]:
train_dmatrix = xgb.DMatrix(data=X, label=y)

/Users/JohnSchueler/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:173: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [18]:
params = {"objective":"reg:logistic", "mkax_depth":3}

In [19]:
cv_results = xgb.cv(dtrain=train_dmatrix, params=params,
                   nfold=3, num_boost_round=5,
                   metrics="error", as_pandas=True, seed=123)

[20:21:10] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { mkax_depth } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:21:10] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { mkax_depth } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:21:10] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { mkax_depth } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but sl

In [20]:
print(cv_results)

   train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.002055         0.000839         0.002055        0.001677
1          0.002055         0.000839         0.002055        0.001677
2          0.002055         0.000839         0.002055        0.001677
3          0.002055         0.000839         0.002055        0.001677
4          0.002055         0.000839         0.002055        0.001677


In [21]:
print(((1-cv_results["test-error-mean"]).iloc[-1]))

0.997945
